<a href="https://colab.research.google.com/github/magnuum/data_science/blob/main/corona_science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Wczytanie potrzebnych bibliotek
import pandas as pd
import plotly.express as px
import datetime
import numpy as np
from sklearn.linear_model import LinearRegression

#Pobranie danych z wybranego pliku csv, zawierającego dane o koronawirusie na świecie
url= f"https://covid.ourworldindata.org/data/ecdc/full_data.csv"
df = pd.read_csv(url, error_bad_lines=False)

In [ ]:
#Funkcja służąca do ustawienia odpowiedniego formatu daty
#Przykładowe wywołanie: format_data(datetime.date(2020,3,26))
def format_data(date: datetime.date):
  return date.strftime('%Y-%#m-%#d')

In [ ]:
#Funkcja pokazująca na mapie świata przyrost śmierci w poszczególnych krajach na skutek COVID-19
#Przykładowe wywołanie: new_deaths(datetime.date(2020,3,26))
def new_deaths(date: datetime.date):

  #Ustawienie daty w odpowiednim formacie 
  data = format_data(date)

  #Wybranie danych z tabeli o odpowiedniej dacie i pogrupowanie ich pod wzgędam krajów, oraz wyrzucenie wiersza z danymi o świecie
  formated_gdf = df.loc[df["date"]==data].groupby("location").max().drop("World")
  formated_gdf = formated_gdf.reset_index()

  #Stworzenie definicji figury mapy świata z danymi o nowych śmierciach
  fig = px.choropleth(formated_gdf, locations="location", locationmode='country names', 
                     color="new_deaths", hover_name="location", 
                     range_color= [0, max(formated_gdf['new_deaths'])], 
                     projection="natural earth", color_continuous_scale=px.colors.sequential.Cividis_r,  
                     title='Przyrost przypadków śmierci na skutek COVID-19 na swiecie')
  
  #Wyświetlenie mapy
  fig.show()


In [ ]:
#Wywołanie funkcji new_deaths
new_deaths(datetime.date(2020,3,26))

In [ ]:
#Funkcja pokazująca na mapie świata przyrost przypadków zachorowań na COVID-19 w poszczególnych krajach
#Przykładowe wywołanie: new_cases(datetime.date(2020,3,26))
def new_cases(date: datetime.date):

  #Ustawienie daty w odpowiednim formacie 
  data = format_data(date)

  #Wybranie danych z tabeli o odpowiedniej dacie i pogrupowanie ich pod wzgędam krajów, oraz wyrzucenie wiersza z danymi o świecie
  formated_gdf = df.loc[df["date"]==data].groupby("location").max().drop("World")
  formated_gdf = formated_gdf.reset_index()

  #Stworzenie definicji figury mapy świata z danymi o nowych przypadkach zarażenia
  fig = px.choropleth(formated_gdf, locations="location", locationmode='country names', 
                     color="new_cases", hover_name="location", 
                     range_color= [0, max(formated_gdf['new_cases'])], 
                     projection="natural earth", color_continuous_scale=px.colors.sequential.Cividis_r,  
                     title='Przyrost przypadków zachorowań na COVID-19 na swiecie')
  
  #Wyświetlenie mapy
  fig.show()

In [ ]:
#Wywołanie funkcji new_cases
new_cases(datetime.date(2020,3,26))

In [ ]:
#Funkcja pokazująca na mapie świata stosunek śmierci do zachorowań na skutek COVID-19 
#Przykładowe wywołanie: deaths_to_cases(datetime.date(2020,3,26))
def deaths_to_cases(date: datetime.date):

  #Ustawienie daty w odpowiednim formacie
  data = format_data(date)

  #Wybranie danych z tabeli o odpowiedniej dacie i pogrupowanie ich pod wzgędam krajów, oraz wyrzucenie wiersza z danymi o świecie
  formated_gdf = df.loc[df["date"]==data].groupby("location").max().drop("World")
  formated_gdf = formated_gdf.reset_index()

  #Dodanie do DataFramu nowej kolumny z danymi stosunku śmierci do przypadków
  formated_gdf["proportion"] = formated_gdf["total_cases"]/formated_gdf["total_deaths"]

  #Stworzenie definicji figury mapy świata z danymi o stosunku śmierci do przypadków
  fig = px.choropleth(formated_gdf, locations="location", locationmode='country names', 
                     color="proportion", hover_name="location", 
                     range_color= [0, max(formated_gdf['proportion'])], 
                     projection="natural earth", color_continuous_scale=px.colors.sequential.Cividis_r,  
                     title='Stosunek śmierci do przypadków COVID19 na swiecie')
  
  #Wyświetlenie mapy
  fig.show()

In [ ]:
#Wywołanie funkcji new_cases
deaths_to_cases(datetime.date(2020,3,26))

In [ ]:
#Funkcja zwracająca tabelę z oszacowanymi danymi na temat śmierci i zarażonych na COVID-19 na najbliższe 5 dni w danym kraju 
#Przykładowe wywołanie: predict_cases("Italy")
def predict_cases(country):

  #Utworznie DataFramu z danymi z ostatnich 10 dni w danym kraju
  frame = df.loc[df["location"]==country].tail(10).drop("location", axis=1).reset_index(drop=True)

  #Utworzenie danych treningowych potrzebnych do modelu regresji liniowej
  x = []
  for i,row in frame.iterrows():
    x.append(i)
  y_t_cases = frame["total_cases"]
  y_t_deaths = frame["total_deaths"]
  y_n_cases = frame["new_cases"]
  y_n_deaths = frame["new_deaths"]
  x, y_t_cases, y_t_deaths, y_n_cases, y_n_deaths = np.array(x), np.array(y_t_cases), np.array(y_t_deaths), np.array(y_n_cases), np.array(y_n_deaths)
  x = x.reshape((-1, 1))

  #Stworzenie modelu regresji linowej dla poszczególnych danych
  model_t_cases = LinearRegression().fit(x, y_t_cases)
  model_t_deaths = LinearRegression().fit(x, y_t_deaths)
  model_n_cases = LinearRegression().fit(x, y_n_cases)
  model_n_deaths = LinearRegression().fit(x, y_n_deaths)

  #Kolejne 5 dni
  x_new = [10, 11, 12, 13, 14]
  x_new = np.array(x_new)
  x_new = x_new.reshape((-1, 1))

  #Wczytanie ostatniej daty z DataFramu
  dzien = frame["date"].values[9]
  date_time_obj = datetime.datetime.strptime(dzien, '%Y-%m-%d')

  #Stworzenie serii danych z datą kolejnych 5 dni
  data_new = []
  for i in range(1,6):
    di = date_time_obj+datetime.timedelta(days=i)
    di = format_data(di)
    data_new.append(di)

  #Stworzenie spodziewanych danych na kolejne 5 dni przy pomocy modelu liniowej regresji  
  y_pred_new_t_cases = model_t_cases.predict(x_new).round()
  y_pred_new_t_deaths = model_t_deaths.predict(x_new).round()
  y_pred_new_n_cases = model_n_cases.predict(x_new).round()
  y_pred_new_n_deaths = model_n_deaths.predict(x_new).round()

  #Stworzenie nowego DataFramu z spodziewanymi danymi na kolejne 5 dni
  d = {"date":data_new, "new_cases":y_pred_new_n_cases, "new_deaths": y_pred_new_n_deaths, "total_cases":y_pred_new_t_cases,
       "total_deaths":y_pred_new_t_deaths}
  frame_pred = pd.DataFrame(data=d)  

  #Dodanie DataFramu z spodziewanymi danymi na kolejne 5 dni do DatFramu z danymi z poprzednich dni
  new_frame = frame.append(frame_pred).reset_index(drop=True)   

  return new_frame

In [ ]:
predict_cases("Italy")

,date,new_cases,new_deaths,total_cases,total_deaths
0,2020-03-17,4000.0,347.0,27980.0,2158.0
1,2020-03-18,3526.0,347.0,31506.0,2505.0
2,2020-03-19,4207.0,473.0,35713.0,2978.0
3,2020-03-20,5322.0,429.0,41035.0,3407.0
4,2020-03-21,5986.0,625.0,47021.0,4032.0
5,2020-03-22,6557.0,795.0,53578.0,4827.0
6,2020-03-23,5560.0,649.0,59138.0,5476.0
7,2020-03-24,4789.0,601.0,63927.0,6077.0
8,2020-03-25,5249.0,743.0,69176.0,6820.0
9,2020-03-26,5210.0,685.0,74386.0,7505.0
